In [ ]:
!pip install transformers datasets evaluate seqeval --quiet

In [ ]:
from datasets import Dataset
import pandas as pd

In [ ]:


def load_conll_to_dataset(filepath):
    tokens_list, labels_list = [], []
    tokens, labels = [], []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == "":
                if tokens:
                    tokens_list.append(tokens)
                    labels_list.append(labels)
                    tokens, labels = [], []
            else:
                token, label = line.split('\t')
                tokens.append(token)
                labels.append(label)
    if tokens:
        tokens_list.append(tokens)
        labels_list.append(labels)
    df = pd.DataFrame({'tokens': tokens_list, 'ner_tags': labels_list})
    return Dataset.from_pandas(df)

dataset = load_conll_to_dataset('/content/labeled_subset.conll')  # adjust path if needed


 STEP 3: Tokenizer Setup

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "xlm-roberta-base"  # or "Davlan/bert-tiny-amharic" or "Davlan/afro-xlmr-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

label_list = ["O", "B-Product", "I-Product", "B-PRICE", "I-PRICE", "B-LOC", "I-LOC"]
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

✂️ STEP 4: Tokenization & Label Alignment

In [ ]:
def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True, padding=True)
    labels = []

    for i, label in enumerate(batch["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            else:
                current_label = label[word_idx]
                if current_label.startswith("B-"):
                    label_ids.append(label_to_id[current_label.replace("B-", "I-")])
                else:
                    label_ids.append(label_to_id[current_label])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

STEP 5: Load Model

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label=id_to_label,
    label2id=label_to_id
)


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


STEP 6: Evaluation Metric

In [ ]:
import evaluate
metric = evaluate.load("seqeval")

def compute_metrics(p):
    import numpy as np
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = []
    true_labels = []
    for pred, label in zip(predictions, labels):
        true_pred = [id_to_label[p] for (p, l) in zip(pred, label) if l != -100]
        true_lab = [id_to_label[l] for l in label if l != -100]
        true_predictions.append(true_pred)
        true_labels.append(true_lab)

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


STEP 7: Set TrainingArguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner_model",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs"
)


STEP 8: Train Using HuggingFace Trainer

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # ideally split this
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-11-4178046100.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dmuay2015 (dmuay2015-debre-markos-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=20, training_loss=0.7042546272277832, metrics={'train_runtime': 50.3555, 'train_samples_per_second': 4.965, 'train_steps_per_second': 0.397, 'total_flos': 5486459203500.0, 'train_loss': 0.7042546272277832, 'epoch': 5.0})

In [ ]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.04524093493819237, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.9957028852056476, 'eval_runtime': 0.3014, 'eval_samples_per_second': 165.892, 'eval_steps_per_second': 13.271, 'epoch': 5.0}


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


STEP 9: Save Model

In [ ]:
trainer.save_model("fine_tuned_ner_amharic")
tokenizer.save_pretrained("fine_tuned_ner_amharic")

('fine_tuned_ner_amharic/tokenizer_config.json',
 'fine_tuned_ner_amharic/special_tokens_map.json',
 'fine_tuned_ner_amharic/sentencepiece.bpe.model',
 'fine_tuned_ner_amharic/added_tokens.json',
 'fine_tuned_ner_amharic/tokenizer.json')

mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

drive_path = "/content/drive/MyDrive/Amharic_NER_Model"
os.makedirs(drive_path, exist_ok=True)

Save Model, Tokenizer, and Config

In [ ]:
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)

('/content/drive/MyDrive/Amharic_NER_Model/tokenizer_config.json',
 '/content/drive/MyDrive/Amharic_NER_Model/special_tokens_map.json',
 '/content/drive/MyDrive/Amharic_NER_Model/sentencepiece.bpe.model',
 '/content/drive/MyDrive/Amharic_NER_Model/added_tokens.json',
 '/content/drive/MyDrive/Amharic_NER_Model/tokenizer.json')

Save Training Arguments and Metrics

In [ ]:
import json

# Extract only serializable fields
def is_json_serializable(value):
    try:
        json.dumps(value)
        return True
    except (TypeError, OverflowError):
        return False

args_dict = {k: v for k, v in vars(trainer.args).items() if is_json_serializable(v)}

# Save to JSON
with open(f"{drive_path}/training_args.json", "w") as f:
    json.dump(args_dict, f, indent=2)



To Load the Model Later

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

model_path = "/content/drive/MyDrive/Amharic_NER_Model"

model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


Test the Model on New Examples

In [ ]:
from transformers import pipeline

ner_pipeline = pipeline("token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

example = "አዲስ ጫማ ዋጋ 1200 ብር በቦሌ እንደምታገኙ"
results = ner_pipeline(example)

for r in results:
    print(f"{r['word']}: {r['entity_group']} (score: {r['score']:.2f})")


Device set to use cuda:0


In [ ]:
from transformers import pipeline

In [ ]:
# Load the pipeline with GPU support
ner_pipeline = pipeline(
    "token-classification",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",  # groups subwords into single words
    device=0  # cuda:0
)

# Test input
example = "አዲስ ጫማ ዋጋ 1200 ብር በቦሌ እንደምታገኙ"

# Run inference
results = ner_pipeline(example)

# Display results
for r in results:
    print(f"{r['word']}: {r['entity_group']} (score: {r['score']:.2f})")


Device set to use cuda:0
